In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace

scala_version = '2.13'
spark_version = '3.5.5'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}','org.apache.kafka:kafka-clients:2.10.0']
spark = SparkSession.builder.master('local')\
.appName("kafka-example").config("spark.jars.packages", ",".join(packages))\
.config("spark.memory.offHeap.enabled","true")\
.config("spark.memory.offHeap.size","10g")\
.config("spark.driver.memory", "16g")\
.getOrCreate()

In [2]:
topic_name = 'RandomNumberStreaming'
kafka_server = 'localhost:9092'

streamRawDf = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", kafka_server)\
    .option("subscribe", topic_name)\
    .load()

streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string').alias('rand_number'))
checkEvenDF = streamDF.withColumn('Is_Even',(regexp_replace(col('rand_number'), '"', '').cast('int') % 2 == 0))

In [3]:
stream_writer1 = (streamDF.writeStream.queryName("RandomNumberQuery").trigger(processingTime="5 seconds").outputMode("append").format("memory"))
stream_writer2 = (checkEvenDF.writeStream.queryName("EvenCheckRandomNumberQuery").trigger(processingTime="5 seconds").outputMode("append").format("memory"))
query1 = stream_writer1.start()    
query2 = stream_writer2.start()

In [4]:
from time import sleep
from IPython.display import display, clear_output
for x in range(0, 100):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        result1 = spark.sql(f"SELECT * FROM {query1.name}")
        result2 = spark.sql(f"SELECT * FROM {query2.name}")
        display(result1.toPandas())
        display(result2.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")


Showing live view refreshed every 5 seconds
Seconds passed: 35


,topic,offset,rand_number
0,RandomNumberStreaming,59,"""40"""
1,RandomNumberStreaming,60,"""5"""
2,RandomNumberStreaming,61,"""1"""
3,RandomNumberStreaming,62,"""43"""
4,RandomNumberStreaming,63,"""64"""
5,RandomNumberStreaming,64,"""30"""
6,RandomNumberStreaming,65,"""17"""
7,RandomNumberStreaming,66,"""36"""
8,RandomNumberStreaming,67,"""7"""
9,RandomNumberStreaming,68,"""88"""


,topic,offset,rand_number,Is_Even
0,RandomNumberStreaming,59,"""40""",True
1,RandomNumberStreaming,60,"""5""",False
2,RandomNumberStreaming,61,"""1""",False
3,RandomNumberStreaming,62,"""43""",False
4,RandomNumberStreaming,63,"""64""",True
5,RandomNumberStreaming,64,"""30""",True
6,RandomNumberStreaming,65,"""17""",False
7,RandomNumberStreaming,66,"""36""",True
8,RandomNumberStreaming,67,"""7""",False
9,RandomNumberStreaming,68,"""88""",True


break
Live view ended...
